In [1]:
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler

In [2]:
path_data_not = "../22_HP/outputs/simulation_results_nand.csv"
path_data_nor = "../22_HP/outputs/simulation_results_nor.csv"
path_data_xor = "../22_HP/outputs/simulation_results_xor.csv"
path_data_and3 = "../22_HP/outputs/simulation_results_and.csv"
path_data_nand = "../22_HP/outputs/simulation_results_nand.csv"

In [3]:
def load_and_process_data(path):
    data = pd.read_csv(path)
    leaks = data['leakage']
    mean = leaks.mean()
    std = leaks.std()
    return mean, std

not_mean, not_std = load_and_process_data(path_data_not)
and2_mean, and2_std = load_and_process_data(path_data_nor)
or2_mean, or2_std = load_and_process_data(path_data_xor)
xor2_mean, xor2_std = load_and_process_data(path_data_and3)
nand2_mean, nand2_std = load_and_process_data(path_data_nand)

In [4]:
def reverse_scale_leakage(df, mean, std):
  df['leakage'] = (df['leakage'] * std) + mean
  return df

In [5]:
df_c499 = pd.read_csv("../C499/simulation_results_c499.csv")

df_params = df_c499.drop(columns=['leakage'])
df_params.shape[0]

not_inputs = 1
df_not_leakage = df_params
df_not_leakage = pd.concat([df_not_leakage]*(2**not_inputs), ignore_index=True)
df_not_leakage.insert(0, 'vin', 0)
# Conditional statements to assign input values based on index 
df_not_leakage.loc[(df_not_leakage.index // df_params.shape[0]) % 2 == 1, 'vin'] = 1

and_3_inputs = 3
df_and3_leakage = df_params.copy()
df_and3_leakage = pd.concat([df_and3_leakage] * (2**and_3_inputs), ignore_index=True)
df_and3_leakage.insert(0, 'vin1', 0)
df_and3_leakage.insert(1, 'vin2', 0)
df_and3_leakage.insert(2, 'vin3', 0)
df_and3_leakage.loc[(df_and3_leakage.index // df_params.shape[0]) % 2 == 1, 'vin1'] = 1

xor_2_inputs = 2
df_xor2_leakage = df_params
df_xor2_leakage = pd.concat([df_xor2_leakage]*(2**xor_2_inputs), ignore_index=True)
df_xor2_leakage.insert(0, 'vina', 0)
df_xor2_leakage.insert(1, 'vinb', 0)
df_xor2_leakage.loc[(df_xor2_leakage.index // df_params.shape[0]) % 2 == 1, 'vina'] = 1
df_xor2_leakage.loc[(df_xor2_leakage.index // df_params.shape[0]) % 4 >= 2, 'vinb'] = 1

nand_2_inputs = 2
df_nand2_leakage = df_params
df_nand2_leakage = pd.concat([df_nand2_leakage]*(2**nand_2_inputs), ignore_index=True)
df_nand2_leakage.insert(0, 'vina', 0)
df_nand2_leakage.insert(1, 'vinb', 0)
df_nand2_leakage.loc[(df_nand2_leakage.index // df_params.shape[0]) % 2 == 1, 'vina'] = 1
df_nand2_leakage.loc[(df_nand2_leakage.index // df_params.shape[0]) % 4 >= 2, 'vinb'] = 1

nor_2_inputs = 2
df_nor2_leakage = df_params
df_nor2_leakage = pd.concat([df_nor2_leakage]*(2**nor_2_inputs), ignore_index=True)
df_nor2_leakage.insert(0, 'vina', 0)
df_nor2_leakage.insert(1, 'vinb', 0)
df_nor2_leakage.loc[(df_nor2_leakage.index // df_params.shape[0]) % 2 == 1, 'vina'] = 1
df_nor2_leakage.loc[(df_nor2_leakage.index // df_params.shape[0]) % 4 >= 2, 'vinb'] = 1


In [6]:
def not_inference(): 
  model = pickle.load(open("models/not_catboost.pkl", 'rb'))
  df_send = df_not_leakage.copy()
  scaler = StandardScaler()
  scaled_data = scaler.fit_transform(df_send)
  df_scaled = pd.DataFrame(scaled_data, columns=df_not_leakage.columns)
  predictions = model.predict(df_scaled)
  df_predictions = pd.DataFrame(predictions, columns=['leakage'])
  df_predictions = reverse_scale_leakage(df_predictions, not_mean, not_std)
  df_predictions = pd.concat([df_send, df_predictions], axis=1)
  return df_predictions

In [7]:
def nand_inference():
  model = pickle.load(open("models/nand_catboost.pkl", 'rb'))
  df_send = df_and3_leakage.copy()
  scaler = StandardScaler()
  scaled_data = scaler.fit_transform(df_send)
  df_scaled = pd.DataFrame(scaled_data, columns=df_and3_leakage.columns)
  predictions = model.predict(df_scaled)
  df_predictions = pd.DataFrame(predictions, columns=['leakage'])
  df_predictions = reverse_scale_leakage(df_predictions, and2_mean, and2_std)
  df_predictions = pd.concat([df_send, df_predictions], axis=1)
  return df_predictions

In [8]:
def and_inference():
  model = pickle.load(open("models/and_catboost.pkl", 'rb'))
  df_send = df_and3_leakage.copy()
  scaler = StandardScaler()
  scaled_data = scaler.fit_transform(df_send)
  df_scaled = pd.DataFrame(scaled_data, columns=df_and3_leakage.columns)
  predictions = model.predict(df_scaled)
  df_predictions = pd.DataFrame(predictions, columns=['leakage'])
  df_predictions = reverse_scale_leakage(df_predictions, and2_mean, and2_std)
  df_predictions = pd.concat([df_send, df_predictions], axis=1)
  return df_predictions

In [9]:
def nor_inference():
  model = pickle.load(open("models/nor_catboost.pkl", 'rb'))
  df_send = df_nor2_leakage.copy()
  scaler = StandardScaler()
  scaled_data = scaler.fit_transform(df_send)
  df_scaled = pd.DataFrame(scaled_data, columns=df_nor2_leakage.columns)
  predictions = model.predict(df_scaled)
  df_predictions = pd.DataFrame(predictions, columns=['leakage'])
  df_predictions = reverse_scale_leakage(df_predictions, or2_mean, or2_std)
  df_predictions = pd.concat([df_send, df_predictions], axis=1)
  return df_predictions

In [10]:
def xor_inference():
  model = pickle.load(open("models/xor_catboost.pkl", 'rb'))
  df_send = df_xor2_leakage.copy()
  scaler = StandardScaler()
  scaled_data = scaler.fit_transform(df_send)
  df_scaled = pd.DataFrame(scaled_data, columns=df_xor2_leakage.columns)
  predictions = model.predict(df_scaled)
  df_predictions = pd.DataFrame(predictions, columns=['leakage'])
  df_predictions = reverse_scale_leakage(df_predictions, xor2_mean, xor2_std)
  df_predictions = pd.concat([df_send, df_predictions], axis=1)
  return df_predictions

In [11]:
not_predictions = not_inference()
and_predictions = and_inference()
nor_predictions = nor_inference()
xor_predictions = xor_inference()
nand_predictions = nand_inference()

In [22]:
def not_leakage(A, i):
    index = int(10 * A + i)
    return not_predictions.iloc[index]['leakage']

# check 
def and3_leakage(A, B, C, i):
  index = int(A * 20 + B * 10 + C * 10 + i)
  return and_predictions.iloc[index]['leakage']


def xor_leakage(A, B, i):
  index = int(20 * A + 10 * B + i)
  return xor_predictions.iloc[index]['leakage']

def nor_leakage(A, B, i):
  index = int(20 * A + 10 * B + i)
  return nor_predictions.iloc[index]['leakage']

def nand_leakage(A, B, i):
  index = int(20 * A + 10 * B + i)
  return nand_predictions.iloc[index]['leakage']


In [23]:
id0 = 1
id1 = 1
id2 = 1
id3 = 1
id4 = 1
id5 = 1
id6 = 1
id7 = 1
id8 = 1
id9 = 1
id10 = 1
id11 = 1
id12 = 1
id13 = 1
id14 = 1
id15 = 1
id16 = 1
id17 = 1
id18 = 1
id19 = 1
id20 = 1
id21 = 1
id22 = 1
id23 = 1
id24 = 1
id25 = 1
id26 = 1
id27 = 1
id28 = 1
id29 = 1
id30 = 1
id31 = 1

ic0 = 1
ic1 = 1
ic2 = 1
ic3 = 1
ic4 = 1
ic5 = 1
ic6 = 1
ic7 = 1

r = 1

In [26]:
def run(i):
  leakage = 0

  xa0 = id0 ^ id1
  leakage += xor_leakage(id0, id1, i)

  xa1 = id2 ^ id3
  leakage += xor_leakage(id2, id3, i)

  xa2 = id4 ^ id5
  leakage += xor_leakage(id4, id5, i)

  xa3 = id6 ^ id7
  leakage += xor_leakage(id6, id7, i)

  xa4 = id8 ^ id9
  leakage += xor_leakage(id8, id9, i)

  xa5 = id10 ^ id11
  leakage += xor_leakage(id10, id11, i)

  xa6 = id12 ^ id13
  leakage += xor_leakage(id12, id13, i)

  xa7 = id14 ^ id15
  leakage += xor_leakage(id14, id15, i)

  xa8 = id16 ^ id17
  leakage += xor_leakage(id16, id17, i)

  xa9 = id18 ^ id19
  leakage += xor_leakage(id18, id19, i)

  xa10 = id20 ^ id21
  leakage += xor_leakage(id20, id21, i)

  xa11 = id22 ^ id23
  leakage += xor_leakage(id22, id23, i)

  xa12 = id24 ^ id25
  leakage += xor_leakage(id24, id25, i)

  xa13 = id26 ^ id27
  leakage += xor_leakage(id26, id27, i)

  xa14 = id28 ^ id29
  leakage += xor_leakage(id28, id29, i)

  xa15 = id30 ^ id31
  leakage += xor_leakage(id30, id31, i)



  f0 = xa0 ^ xa1
  leakage += xor_leakage(xa0, xa1, i)

  f1 = xa2 ^ xa3
  leakage += xor_leakage(xa2, xa3, i)

  f2 = xa4 ^ xa5
  leakage += xor_leakage(xa4, xa5, i)

  f3 = xa6 ^ xa7
  leakage += xor_leakage(xa6, xa7, i)

  f4 = xa8 ^ xa9
  leakage += xor_leakage(xa8, xa9, i)

  f5 = xa10 ^ xa11
  leakage += xor_leakage(xa10, xa11, i)

  f6 = xa12 ^ xa13
  leakage += xor_leakage(xa12, xa13, i)

  f7 = xa14 ^ xa15
  leakage += xor_leakage(xa14, xa15, i)



  h0 = ic0 & r
  leakage += not_leakage(nand_leakage(ic0, r, i), i)

  h1 = ic1 & r
  leakage += not_leakage(nand_leakage(ic1, r, i), i)

  h2 = ic2 & r
  leakage += not_leakage(nand_leakage(ic2, r, i), i)

  h3 = ic3 & r
  leakage += not_leakage(nand_leakage(ic3, r, i), i)

  h4 = ic4 & r
  leakage += not_leakage(nand_leakage(ic4, r, i), i)

  h5 = ic5 & r
  leakage += not_leakage(nand_leakage(ic5, r, i), i)

  h6 = ic6 & r
  leakage += not_leakage(nand_leakage(ic6, r, i), i)

  h7 = ic7 & r
  leakage += not_leakage(nand_leakage(ic7, r, i), i)



  xb0 = id0 ^ id4
  leakage += xor_leakage(id0, id4, i)

  xb1 = id1 ^ id5
  leakage += xor_leakage(id1, id5, i)

  xb2 = id2 ^ id6
  leakage += xor_leakage(id2, id6, i)

  xb3 = id3 ^ id7
  leakage += xor_leakage(id3, id7, i)

  xb4 = id16 ^ id20
  leakage += xor_leakage(id16, id20, i)

  xb5 = id17 ^ id21
  leakage += xor_leakage(id17, id21, i)

  xb6 = id18 ^ id22
  leakage += xor_leakage(id18, id22, i)

  xb7 = id19 ^ id23
  leakage += xor_leakage(id19, id23, i)



  xc0 = id8 ^ id12
  leakage += xor_leakage(id8, id12, i)

  xc1 = id9 ^ id13
  leakage += xor_leakage(id9, id13, i)

  xc2 = id10 ^ id14
  leakage += xor_leakage(id10, id14, i)

  xc3 = id11 ^ id15
  leakage += xor_leakage(id11, id15, i)

  xc4 = id24 ^ id28
  leakage += xor_leakage(id24, id28, i)

  xc5 = id25 ^ id29
  leakage += xor_leakage(id25, id29, i)

  xc6 = id26 ^ id30
  leakage += xor_leakage(id26, id30, i)

  xc7 = id27 ^ id31
  leakage += xor_leakage(id27, id31, i)



  xe0 = xb0 ^ xc0
  leakage += xor_leakage(xb0, xc0, i)

  xe1 = xb1 ^ xc1
  leakage += xor_leakage(xb1, xc1, i)

  xe2 = xb2 ^ xc2
  leakage += xor_leakage(xb2, xc2, i)

  xe3 = xb3 ^ xc3
  leakage += xor_leakage(xb3, xc3, i)

  xe4 = xb4 ^ xc4
  leakage += xor_leakage(xb4, xc4, i)

  xe5 = xb5 ^ xc5
  leakage += xor_leakage(xb5, xc5, i)

  xe6 = xb6 ^ xc6
  leakage += xor_leakage(xb6, xc6, i)

  xe7 = xb7 ^ xc7
  leakage += xor_leakage(xb7, xc7, i)



  g0 = f0 ^ f1
  leakage += xor_leakage(f0, f1, i)

  g1 = f2 ^ f3
  leakage += xor_leakage(f2, f3, i)

  g2 = f0 ^ f2
  leakage += xor_leakage(f0, f2, i)

  g3 = f1 ^ f3
  leakage += xor_leakage(f1, f3, i)

  g4 = f4 ^ f5
  leakage += xor_leakage(f4, f5, i)

  g5 = f6 ^ f7
  leakage += xor_leakage(f6, f7, i)

  g6 = f4 ^ f6
  leakage += xor_leakage(f4, f6, i)

  g7 = f5 ^ f7
  leakage += xor_leakage(f5, f7, i)



  xd0 = g4 ^ h0
  leakage += xor_leakage(g4, h0, i)

  xd1 = g5 ^ h1
  leakage += xor_leakage(g5, h1, i)

  xd2 = g6 ^ h2
  leakage += xor_leakage(g6, h2, i)

  xd3 = g7 ^ h3
  leakage += xor_leakage(g7, h3, i)

  xd4 = g0 ^ h4
  leakage += xor_leakage(g0, h4, i)

  xd5 = g1 ^ h5
  leakage += xor_leakage(g1, h5, i)

  xd6 = g2 ^ h6
  leakage += xor_leakage(g2, h6, i)

  xd7 = g3 ^ h7
  leakage += xor_leakage(g3, h7, i)



  s0 = xd0 ^ xe0
  leakage += xor_leakage(xd0, xe0, i)

  s1 = xd1 ^ xe1
  leakage += xor_leakage(xd1, xe1, i)

  s2 = xd2 ^ xe2
  leakage += xor_leakage(xd2, xe2, i)

  s3 = xd3 ^ xe3
  leakage += xor_leakage(xd3, xe3, i)

  s4 = xd4 ^ xe4
  leakage += xor_leakage(xd4, xe4, i)

  s5 = xd5 ^ xe5
  leakage += xor_leakage(xd5, xe5, i)

  s6 = xd6 ^ xe6
  leakage += xor_leakage(xd6, xe6, i)

  s7 = xd7 ^ xe7
  leakage += xor_leakage(xd7, xe7, i)

  # Correction

  s0b0 = not s0
  leakage += not_leakage(s0, i)

  s0b1 = not s0
  leakage += not_leakage(s0, i)

  s0b2 = not s0
  leakage += not_leakage(s0, i)

  s0b3 = not s0
  leakage += not_leakage(s0, i)

  s0b4 = not s0
  leakage += not_leakage(s0, i)

  s1b0 = not s1
  leakage += not_leakage(s1, i)

  s1b1 = not s1
  leakage += not_leakage(s1, i)

  s1b2 = not s1
  leakage += not_leakage(s1, i)

  s1b3 = not s1
  leakage += not_leakage(s1, i)

  s1b4 = not s1
  leakage += not_leakage(s1, i)

  s2b0 = not s2
  leakage += not_leakage(s2, i)

  s2b1 = not s2
  leakage += not_leakage(s2, i)

  s2b2 = not s2
  leakage += not_leakage(s2, i)

  s2b3 = not s2
  leakage += not_leakage(s2, i)

  s2b4 = not s2
  leakage += not_leakage(s2, i)

  s3b0 = not s3
  leakage += not_leakage(s3, i)

  s3b1 = not s3
  leakage += not_leakage(s3, i)

  s3b2 = not s3
  leakage += not_leakage(s3, i)

  s3b3 = not s3
  leakage += not_leakage(s3, i)

  s3b4 = not s3
  leakage += not_leakage(s3, i)

  s4b0 = not s4
  leakage += not_leakage(s4, i)

  s4b1 = not s4
  leakage += not_leakage(s4, i)

  s4b2 = not s4
  leakage += not_leakage(s4, i)

  s4b3 = not s4
  leakage += not_leakage(s4, i)

  s4b4 = not s4
  leakage += not_leakage(s4, i)

  s5b0 = not s5
  leakage += not_leakage(s5, i)

  s5b1 = not s5
  leakage += not_leakage(s5, i)

  s5b2 = not s5
  leakage += not_leakage(s5, i)

  s5b3 = not s5
  leakage += not_leakage(s5, i)

  s5b4 = not s5
  leakage += not_leakage(s5, i)

  s6b0 = not s6
  leakage += not_leakage(s6, i)

  s6b1 = not s6
  leakage += not_leakage(s6, i)

  s6b2 = not s6
  leakage += not_leakage(s6, i)

  s6b3 = not s6
  leakage += not_leakage(s6, i)

  s6b4 = not s6
  leakage += not_leakage(s6, i)

  s7b0 = not s7
  leakage += not_leakage(s7, i)

  s7b1 = not s7
  leakage += not_leakage(s7, i)

  s7b2 = not s7
  leakage += not_leakage(s7, i)

  s7b3 = not s7
  leakage += not_leakage(s7, i)

  s7b4 = not s7
  leakage += not_leakage(s7, i)



  t0 = s0b0 & s1b0 & s2b0 & s3
  leakage += not_leakage(nand_leakage(s0b0, s1b0, i), i)
  leakage += not_leakage(nand_leakage(s2b0, s3, i), i)
  leakage += not_leakage(nand_leakage(nand_leakage(s0b0, s1b0, i), nand_leakage(s2b0, s3, i), i), i)

  t1 = s0b1 & s1b1 & s2 & s3b1
  leakage += not_leakage(nand_leakage(s0b1, s1b1, i), i)
  leakage += not_leakage(nand_leakage(s2, s3b1, i), i)
  leakage += not_leakage(nand_leakage(nand_leakage(s0b1, s1b1, i), nand_leakage(s2, s3b1, i), i), i)

  t2 = t2 = s0b2 & s1 & s2b1 & s3b2
  leakage += not_leakage(nand_leakage(s0b2, s1, i), i)
  leakage += not_leakage(nand_leakage(s2b1, s3b2, i), i)
  leakage += not_leakage(nand_leakage(nand_leakage(s0b2, s1, i), nand_leakage(s2b1, s3b2, i), i), i)

  t3 = s0 & s1b2 & s2b2 & s3b3
  leakage += not_leakage(nand_leakage(s0, s1b2, i), i)
  leakage += not_leakage(nand_leakage(s2b2, s3b3, i), i)
  leakage += not_leakage(nand_leakage(nand_leakage(s0, s1b2, i), nand_leakage(s2b2, s3b3, i), i), i)

  t4 = s4b0 & s5b0 & s6b0 & s7
  leakage += not_leakage(nand_leakage(s4b0, s5b0, i), i)
  leakage += not_leakage(nand_leakage(s6b0, s7, i), i)
  leakage += not_leakage(nand_leakage(nand_leakage(s4b0, s5b0, i), nand_leakage(s6b0, s7, i), i), i)

  t5 = s4b1 & s5b1 & s6 & s7b0
  leakage += not_leakage(nand_leakage(s4b1, s5b1, i), i)
  leakage += not_leakage(nand_leakage(s6, s7b0, i), i)
  leakage += not_leakage(nand_leakage(nand_leakage(s4b1, s5b1, i), nand_leakage(s6, s7b0, i), i), i)

  t6 = s4b2 & s5 & s6b1 & s7b1
  leakage += not_leakage(nand_leakage(s4b2, s5, i), i)
  leakage += not_leakage(nand_leakage(s6b1, s7b1, i), i)
  leakage += not_leakage(nand_leakage(nand_leakage(s4b2, s5, i), nand_leakage(s6b1, s7b1, i), i), i)

  t7 = s4 & s5b2 & s6b2 & s7b2
  leakage += not_leakage(nand_leakage(s4, s5b2, i), i)
  leakage += not_leakage(nand_leakage(s6b2, s7b2, i), i)
  leakage += not_leakage(nand_leakage(nand_leakage(s4, s5b2, i), nand_leakage(s6b2, s7b2, i), i), i)



  u0 = t0 | t1 | t2 | t3
  leakage += not_leakage(nor_leakage(t0, t1, i), i)
  leakage += not_leakage(nor_leakage(t2, t3, i), i)
  leakage += not_leakage(nor_leakage(t0|t1, t2|t3, i), i)

  u1 = t4 | t5 | t6 | t7
  leakage += not_leakage(nor_leakage(t4, t5, i), i)
  leakage += not_leakage(nor_leakage(t6, t7, i), i)
  leakage += not_leakage(nor_leakage(t4|t5, t6|t7, i), i)



  w0 = s4 & s5b3 & s6 & s7b3 & u0
  leakage += not_leakage(nand_leakage(s4, s5b3, i), i)
  leakage += not_leakage(nand_leakage(s6, s7b3, i), i)
  leakage += not_leakage(nand_leakage(s4&s5b3, u0, i), i)
  leakage += not_leakage(nand_leakage(s6&s7b3, s4&s5b3&u0, i), i)

  w1 = s4 & s5b4 & s6b3 & s7 & u0
  leakage += not_leakage(nand_leakage(s4, s5b4, i), i)
  leakage += not_leakage(nand_leakage(s6b3, s7, i), i)
  leakage += not_leakage(nand_leakage(s4&s5b4, u0, i), i)
  leakage += not_leakage(nand_leakage(s6b3&s7, s4&s5b4&u0, i), i)

  w2 = s4b3 & s5 & s6 & s7b4 & u0
  leakage += not_leakage(nand_leakage(s4b3, s5, i), i)
  leakage +=not_leakage(nand_leakage(s6, s7b4, i), i)
  leakage += not_leakage(nand_leakage(s4b3&s5, u0, i), i)
  leakage += not_leakage(nand_leakage(s6&s7b4, s4b3&s5&u0, i), i)

  w3 = s4b4 & s5 & s6b4 & s7 & u0
  leakage += not_leakage(nand_leakage(s4b4, s5, i), i)
  leakage += not_leakage(nand_leakage(s6b4, s7, i), i)
  leakage += not_leakage(nand_leakage(s4b4&s5, u0, i), i)
  leakage += not_leakage(nand_leakage(s6b4&s7, s4b4&s5&u0, i), i)

  w4 = s0 & s1b3 & s2 & s3b3 & u1
  leakage += not_leakage(nand_leakage(s0, s1b3, i), i)
  leakage += not_leakage(nand_leakage(s2, s3b3, i), i)
  leakage += not_leakage(nand_leakage(s0&s1b3, u1, i), i)
  leakage += not_leakage(nand_leakage(s2&s3b3, s0&s1b3&u1, i), i)

  w5 = s0 & s1b4 & s2b3 & s3 & u1
  leakage += not_leakage(nand_leakage(s0, s1b4, i), i)
  leakage += not_leakage(nand_leakage(s2b3, s3, i), i)
  leakage += not_leakage(nand_leakage(s0&s1b4, u1, i), i)
  leakage += not_leakage(nand_leakage(s2b3&s3, s0&s1b4&u1, i), i)

  w6 = s0b3 & s1 & s2 & s3b4 & u1
  leakage += not_leakage(nand_leakage(s0b3, s1, i), i)
  leakage += not_leakage(nand_leakage(s2, s3b4, i), i)
  leakage += not_leakage(nand_leakage(s0b3&s1, u1, i), i)
  leakage += not_leakage(nand_leakage(s2&s3b4, s0b3&s1&u1, i), i)

  w7 = s0b4 & s1 & s2b4 & s3 & u1
  leakage += not_leakage(nand_leakage(s0b4, s1, i), i)
  leakage += not_leakage(nand_leakage(s2b4, s3, i), i)
  leakage += not_leakage(nand_leakage(s0b4&s1, u1, i), i)
  leakage += not_leakage(nand_leakage(s2b4&s3, s0b4&s1&u1, i), i)



  leakage += not_leakage(nand_leakage(w0, s0, i), i)
  e0 = w0 & s0

  leakage += not_leakage(nand_leakage(w0, s1, i), i)
  e1 = w0 & s1

  leakage += not_leakage(nand_leakage(w0, s2, i), i)
  e2 = w0 & s2

  leakage += not_leakage(nand_leakage(w0, s3, i), i)
  e3 = w0 & s3

  leakage += not_leakage(nand_leakage(w1, s0, i), i)
  e4 = w1 & s0

  leakage += not_leakage(nand_leakage(w1, s1, i), i)
  e5 = w1 & s1

  leakage += not_leakage(nand_leakage(w1, s2, i), i)
  e6 = w1 & s2

  leakage += not_leakage(nand_leakage(w1, s3, i), i)
  e7 = w1 & s3

  leakage += not_leakage(nand_leakage(w2, s0, i), i)
  e8 = w2 & s0

  leakage +=not_leakage(nand_leakage(w2, s1, i), i)
  e9 = w2 & s1

  leakage +=not_leakage(nand_leakage(w2, s2, i), i)
  e10 = w2 & s2

  leakage +=not_leakage(nand_leakage(w2, s3, i), i)
  e11 = w2 & s3

  leakage +=not_leakage(nand_leakage(w3, s0, i), i)
  e12 = w3 & s0

  leakage +=not_leakage(nand_leakage(w3, s1, i), i)
  e13 = w3 & s1

  leakage +=not_leakage(nand_leakage(w3, s2, i), i)
  e14 = w3 & s2

  leakage +=not_leakage(nand_leakage(w3, s3, i), i)
  e15 = w3 & s3

  leakage +=not_leakage(nand_leakage(w4, s4, i), i)
  e16 = w4 & s4

  leakage +=not_leakage(nand_leakage(w4, s5, i), i)
  e17 = w4 & s5

  leakage +=not_leakage(nand_leakage(w4, s6, i), i)
  e18 = w4 & s6

  leakage +=not_leakage(nand_leakage(w4, s7, i), i)
  e19 = w4 & s7

  leakage +=not_leakage(nand_leakage(w5, s4, i), i)
  e20 = w5 & s4

  leakage +=not_leakage(nand_leakage(w5, s5, i), i)
  e21 = w5 & s5

  leakage +=not_leakage(nand_leakage(w5, s6, i), i)
  e22 = w5 & s6

  leakage +=not_leakage(nand_leakage(w5, s7, i), i)
  e23 = w5 & s7

  leakage +=not_leakage(nand_leakage(w6, s4, i), i)
  e24 = w6 & s4

  leakage +=not_leakage(nand_leakage(w6, s5, i), i)
  e25 = w6 & s5

  leakage +=not_leakage(nand_leakage(w6, s6, i), i)
  e26 = w6 & s6

  leakage +=not_leakage(nand_leakage(w6, s7, i), i)
  e27 = w6 & s7

  leakage +=not_leakage(nand_leakage(w7, s4, i), i)
  e28 = w7 & s4

  leakage +=not_leakage(nand_leakage(w7, s5, i), i)
  e29 = w7 & s5

  leakage +=not_leakage(nand_leakage(w7, s6, i), i)
  e30 = w7 & s6

  leakage +=not_leakage(nand_leakage(w7, s7, i), i)
  e31 = w7 & s7



  leakage += xor_leakage(id0, e0, i)
  od0 = id0 ^ e0

  leakage += xor_leakage(id1, e1, i)
  od1 = id1 ^ e1

  leakage += xor_leakage(id2, e2, i)
  od2 = id2 ^ e2

  leakage += xor_leakage(id3, e3, i)
  od3 = id3 ^ e3

  leakage += xor_leakage(id4, e4, i)
  od4 = id4 ^ e4

  leakage += xor_leakage(id5, e5, i)
  od5 = id5 ^ e5

  leakage += xor_leakage(id6, e6, i)
  od6 = id6 ^ e6

  leakage += xor_leakage(id7, e7, i)
  od7 = id7 ^ e7

  leakage += xor_leakage(id8, e8, i)
  od8 = id8 ^ e8

  leakage += xor_leakage(id9, e9, i)
  od9 = id9 ^ e9

  leakage += xor_leakage(id10, e10, i)
  od10 = id10 ^ e10

  leakage += xor_leakage(id11, e11, i)
  od11 = id11 ^ e11

  leakage += xor_leakage(id12, e12, i)
  od12 = id12 ^ e12

  leakage += xor_leakage(id13, e13, i)
  od13 = id13 ^ e13

  leakage += xor_leakage(id14, e14, i)
  od14 = id14 ^ e14

  leakage += xor_leakage(id15, e15, i)
  od15 = id15 ^ e15

  leakage += xor_leakage(id16, e16, i)
  od16 = id16 ^ e16

  leakage += xor_leakage(id17, e17, i)
  od17 = id17 ^ e17

  leakage += xor_leakage(id18, e18, i)
  od18 = id18 ^ e18

  leakage += xor_leakage(id19, e19, i)
  od19 = id19 ^ e19

  leakage += xor_leakage(id20, e20, i)
  od20 = id20 ^ e20

  leakage += xor_leakage(id21, e21, i)
  od21 = id21 ^ e21

  leakage += xor_leakage(id22, e22, i)
  od22 = id22 ^ e22

  leakage += xor_leakage(id23, e23, i)
  od23 = id23 ^ e23

  leakage += xor_leakage(id24, e24, i)
  od24 = id24 ^ e24

  leakage += xor_leakage(id25, e25, i)
  od25 = id25 ^ e25

  leakage += xor_leakage(id26, e26, i)
  od26 = id26 ^ e26

  leakage += xor_leakage(id27, e27, i)
  od27 = id27 ^ e27

  leakage += xor_leakage(id28, e28, i)
  od28 = id28 ^ e28

  leakage += xor_leakage(id29, e29, i)
  od29 = id29 ^ e29

  leakage += xor_leakage(id30, e30, i)
  od30 = id30 ^ e30

  leakage += xor_leakage(id31, e31, i)
  od31 = id31 ^ e31

  return leakage

In [27]:
leakage_pred = []
for i in range(10):
  leakage_pred.append(run(i))

leakage_ground = df_c499["leakage"].to_numpy()

In [37]:
leakage_pred_repeated = []
for i in range(len(leakage_ground) // len(leakage_pred)):
    leakage_pred_repeated.extend(leakage_pred)

# Compute R-squared score
r2 = r2_score(leakage_ground, leakage_pred_repeated)
print("R-squared score:", r2)

R-squared score: -759.0339154926937
